In [1]:
epochs = 1
run_name = "CP-1"
train_path = "/content/drive/MyDrive/Datasets/ReadyCP/train"
test_path = "/content/drive/MyDrive/Datasets/ReadyCP/test"

In [2]:
!pip install -q patchify
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00


In [3]:
!wandb login d9ae8f452cf7f0c4ed803a7364869721e5dd2f52
# token = d9ae8f452cf7f0c4ed803a7364869721e5dd2f52

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# # delete non empty directory
# import shutil

# # Replace 'your_directory' with the actual directory path you want to delete
# directory_path = './files/'

# # Use shutil.rmtree to remove the directory and its contents
# shutil.rmtree(directory_path)

In [6]:
import os

dataset_dir = train_path  # Replace with the path to your dataset folder

# Get the class names from the subfolder names
class_names = [class_name for class_name in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, class_name))]

# Print the class names
print(len(class_names))
print(class_names)

2
['RoughBark', 'StripeCanker']


In [7]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import numpy as np
import cv2
import wandb
from wandb.keras import WandbCallback
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping

In [8]:
class ClassToken(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

In [9]:
def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="gelu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"])(x)
    x = Dropout(cf["dropout_rate"])(x)
    return x

def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(
        num_heads=cf["num_heads"], key_dim=cf["hidden_dim"]
    )(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

In [10]:
def ViT(cf):
    """ Inputs """
    input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    inputs = Input(input_shape)     ## (None, 256, 3072)

    """ Patch + Position Embeddings """
    patch_embed = Dense(cf["hidden_dim"])(inputs)   ## (None, 256, 768)

    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (256, 768)
    embed = patch_embed + pos_embed ## (None, 256, 768)

    """ Adding Class Token """
    token = ClassToken()(embed)
    x = Concatenate(axis=1)([token, embed]) ## (None, 257, 768)

    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)

    """ Classification Head """
    x = LayerNormalization()(x)     ## (None, 257, 768)
    x = x[:, 0, :]
    x = Dense(cf["num_classes"], activation="softmax")(x)

    model = Model(inputs, x)
    return model

custom_objects = {'ClassToken': ClassToken}
# if __name__ == "__main__":
#     config = {}
#     config["num_layers"] = 12
#     config["hidden_dim"] = 768
#     config["mlp_dim"] = 3072
#     config["num_heads"] = 12
#     config["dropout_rate"] = 0.1
#     config["num_patches"] = 256
#     config["patch_size"] = 32
#     config["num_channels"] = 3
#     config["num_classes"] = 5

#     model = ViT(config)
#     model.summary()

In [11]:
""" Hyperparameters """
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 32
hp["lr"] = 1e-4
hp["num_epochs"] = epochs
hp["num_classes"] = len(class_names)
hp["class_names"] = class_names

hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 3072
hp["num_heads"] = 12
hp["dropout_rate"] = 0.1

In [12]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(train_path, test_path):
    split=0.15

    # train dir
    subdirectories = glob(os.path.join(train_path, "*"))
    print("train dir: ", subdirectories)

    # Collect images from each subdirectory
    images = []
    images_array = []
    for subdir in subdirectories:
      jpg_files = [os.path.join(subdir, file) for file in os.listdir(subdir) if file.lower().endswith('.jpg')]
      images_array.extend(jpg_files)

    images = shuffle(images_array)

    print(images)

    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    # train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)


    # test dir
    subdirectories = glob(os.path.join(test_path, "*"))
    print("test dir: ", subdirectories)

    images = []
    images_array = []
    for subdir in subdirectories:
      jpg_files = [os.path.join(subdir, file) for file in os.listdir(subdir) if file.lower().endswith('.jpg')]
      images_array.extend(jpg_files)

    test_x = shuffle(images_array)

    print(test_x)

    return train_x, valid_x, test_x

In [13]:
def process_image_label(path):
    """ Reading images """
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
    image = image/255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])

    # patches = np.reshape(patches, (64, 25, 25, 3))
    # for i in range(64):
    #     cv2.imwrite(f"files/{i}.png", patches[i])

    patches = np.reshape(patches, hp["flat_patches_shape"])
    patches = patches.astype(np.float32)

    """ Label """
    class_name = path.split("/")[-2]
    class_idx = hp["class_names"].index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return patches, class_idx

In [14]:
def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape(hp["flat_patches_shape"])
    labels.set_shape(hp["num_classes"])

    return patches, labels

def tf_dataset(images, batch=32):
    ds = tf.data.Dataset.from_tensor_slices((images))
    ds = ds.map(parse).batch(batch).prefetch(8)
    return ds

In [15]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory for storing files """
create_dir("files")

""" Paths """
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "log.csv")

""" Dataset """
train_x, valid_x, test_x = load_data(train_path, test_path)
print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

train_ds = tf_dataset(train_x, batch=hp["batch_size"])
valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])

train dir:  ['/content/drive/MyDrive/Datasets/ReadyCP/train/RoughBark', '/content/drive/MyDrive/Datasets/ReadyCP/train/StripeCanker']
['/content/drive/MyDrive/Datasets/ReadyCP/train/RoughBark/IMG_4853.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/RoughBark/IMG_E4848.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/StripeCanker/IMG_1678.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/StripeCanker/IMG_1629.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/StripeCanker/IMG_1762.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/StripeCanker/IMG_1875.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/RoughBark/IMG_E4784.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/StripeCanker/IMG_1707.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/StripeCanker/IMG_4877.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/StripeCanker/IMG_1661.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/RoughBark/IMG_E4795.JPG', '/content/drive/MyDrive/Datasets/ReadyCP/train/St

In [16]:
wandb.init(project="tensorflow", name=run_name)

wandb: Currently logged in as: arunwarrier (model-analyze). Use `wandb login --relogin` to force relogin


In [17]:
""" Model """
model = ViT(hp)
model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
    metrics=["acc"]
)

callbacks = [
    ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
    WandbCallback(),
]

model.fit(
    train_ds,
    epochs=hp["num_epochs"],
    validation_data=valid_ds,
    callbacks=callbacks
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


8/8 [==============================] - ETA: 0s - loss: 4.7222 - acc: 0.5232
Epoch 1: val_loss improved from inf to 0.78702, saving model to files/model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231114_110849-3ww5imct/files/model-best)... Done. 145.2s


8/8 [==============================] - 518s 64s/step - loss: 4.7222 - acc: 0.5232 - val_loss: 0.7870 - val_acc: 0.3659 - lr: 1.0000e-04


In [ ]:
wandb.finish()